In [2]:

import torch
import numpy as np
import cv2
import time

import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath


# Load the YOLOv5 model
combined_model = torch.hub.load('ultralytics/yolov5', 'custom', path=r'C:\\Users\\victo\\Desktop\\2 Categories Yolov5.pt',force_reload=True)
combined_model.cuda()
combined_model.amp=False
combined_model.conf = 0.20  # Confidence threshold
combined_model.classes = [0, 1]  # Class 0 for gauze, 1 for hand

# Initialize variables for both cameras
onscreenIn, onscreenOut, countIn, countOut, startTime, endTime, countPlay = 0, 0, 0, 0, time.time(), 0, 0
frameCountIn, frameCountOut = 0, 0
isPaused = False
condition = 7


# Initialize video capture for two cameras
#cap1 = cv2.VideoCapture(0)  # First camera
cap1 = cv2.VideoCapture("C:\\Users\\victo\\Pictures\\Camera Roll\\WIN_20240214_19_38_37_Pro.mp4")  # First Video
cap2 = cv2.VideoCapture(1)  # Second camera

# Display resolution and aspect ratio calculations
display_width_per_camera = 1920 // 2
display_height_per_camera = int(display_width_per_camera * (9 / 16))
display_height = display_height_per_camera

# Initialize the dictionary to track the previous key states
prev_keys = {}

PAUSE_DURATION = 2  # in seconds
UNPAUSE_DURATION = 0
pause_timer = 0  # Timer variable to track the pause duration
unpause_timer = 0 

while True:
    # Read frames from both cameras
    ret, frame1 = cap1.read()
    ret1, frame2 = cap2.read()

    if not ret or not ret1:
        print("Failed to capture frames from cameras")
        break

    frame1 = cv2.resize(frame1, (display_width_per_camera, display_height_per_camera))
    frame2 = cv2.resize(frame2, (display_width_per_camera, display_height_per_camera))

    combined_frame = np.hstack((frame1, frame2))

    # Make detections
    results = combined_model(combined_frame)

    prevOnScreenIn, prevOnScreenOut = onscreenIn, onscreenOut
    a, b, handDetected = 0, 0, False
    # Get the current key press
    key = cv2.waitKey(10) & 0xFF

    # Function to handle key press
    def handle_key_press(key_char, action):
        if prev_keys.get(key_char, -1) != key and key == ord(key_char):
            action()
            prev_keys[key_char] = key
        elif key != ord(key_char):
            prev_keys[key_char] = -1

    # Correcting the lambda functions to properly modify a and b
    handle_key_press('0', lambda: globals().update(countIn=countIn+1))   #a=a+1))
    handle_key_press('1', lambda: globals().update(countIn=countIn-1))   #a=a-1 ))#if a > 0 else 0))
    handle_key_press('2', lambda: globals().update(countOut=countOut+1))   #b=b+1))
    handle_key_press('3', lambda: globals().update(countOut=countOut-1))   #b=b-1 ))#if b > 0 else 0))

    for detection in results.xyxy[0]:
        detected_class = detection[5].item()
        if detected_class == 0:
            if detection[0].item() < frame1.shape[1]:
                a += 1
            else:
                b += 1
        elif detected_class == 1:
            handDetected = True
    
    #isPaused = handDetected
    
    if time.time() - unpause_timer >= UNPAUSE_DURATION:
        isPaused = True
    
    if handDetected:
        pause_timer = time.time()
        isPaused = True
    elif pause_timer > 0 and time.time() - pause_timer >= PAUSE_DURATION:
        isPaused = False
        pause_timer = 0
        unpause_timer = time.time()
    
    
        
    if not isPaused:
        onscreenIn, onscreenOut = a, b
        if onscreenIn > prevOnScreenIn:
            countIn += onscreenIn - prevOnScreenIn
        if onscreenOut > prevOnScreenOut:
            countOut += onscreenOut - prevOnScreenOut
        countPlay = countIn - countOut - onscreenIn
    
    endTime = time.time()
    fps = 1 / (endTime - startTime)
    startTime = endTime

    image = np.squeeze(results.render())

    # Apply text overlays
    cv2.putText(image, f'On Screen In = {onscreenIn}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1, (255, 255, 255), 2)
    cv2.putText(image, f'Total In = {countIn}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX,
                1, (255, 255, 255), 2)
    cv2.putText(image, f'On Screen Out = {onscreenOut}', (display_width_per_camera + 10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(image, f'Total Out = {countOut}', (display_width_per_camera + 10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(image, f'In Play = {countPlay}', (960 - 100, display_height - 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(image, f'FPS = {round(fps, 1)}', (960 - 100, display_height - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    if isPaused:
        cv2.putText(image, 'System Paused - Hand Detected', (960 - 300, display_height // 2),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Gauze Detection', image)

    if key == ord('q'):
        break

cap1.release()
cap2.release()
cv2.destroyAllWindows()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\victo/.cache\torch\hub\master.zip
YOLOv5  2024-3-1 Python-3.11.4 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)

Fusing layers... 
Model summary: 267 layers, 46113663 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


Failed to capture frames from cameras
